In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
from pprint import pprint
import os
import pandas as pd

In [2]:
def init_browser():
    executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
    return Browser("chrome", **executable_path, headless=False)

In [23]:
#scrpe NASA news
def scrape_info():
    browser = init_browser()
    listings = {}

    url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
    browser.visit(url)

    time.sleep(5)
    
    # Scrape page into Soup
    html = browser.html
    soup = bs(html, "html.parser")
    
    results = soup.find_all("div", class_="list_text")
    

    #titles = []
    #bodies = []
    for result in results:
        title = result.find('div', class_='content_title').a.get_text()
        body = result.find('div', class_='article_teaser_body').get_text()
        #titles.append(result.find('div', class_='content_title').a.get_text())
        #bodies.append(result.find('div', class_='article_teaser_body').get_text())
        listings[title] = body

    #listings["news_title"] = titles
    #listings["news_p"] = bodies
    return listings
#pprint(scrape_info())
    

{"10.9 Million Names Now Aboard NASA's Perseverance Mars Rover": 'As part of '
                                                                 "NASA's 'Send "
                                                                 'Your Name to '
                                                                 "Mars' "
                                                                 'campaign, '
                                                                 "they've been "
                                                                 'stenciled '
                                                                 'onto three '
                                                                 'microchips '
                                                                 'along with '
                                                                 'essays from '
                                                                 "NASA's 'Name "
                                                  

In [6]:
#scrpe Mars image data
def image_url():
    browser = init_browser()
    featured_image_url = {}

    image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
    browser.visit(image_url)

    time.sleep(5)

    image_html = browser.html
    image_soup = bs(image_html,"html.parser")

    image_results = image_soup.find_all("div",class_="img")
    
    images = []

    for image_result in image_results:
        images.append(image_result.img["src"])
        #print(image_result.img['src'])
    featured_image_url["image_url"] = images
    return featured_image_url

pprint(image_url())

In [18]:
#scrape Mars_weather variable from twitter#
browser = init_browser()
twitter_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(twitter_url)

time.sleep(5)

twitter_html = browser.html
twitter_soup = bs(twitter_html,"html.parser")

with open("htmltwitter","w") as f:
    f.write(twitter_soup.prettify())

texts = []
for each in twitter_soup.find_all("span",class_="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0"):
    texts.append(each.get_text())
    
long_texts = [text for text in texts if ("InSight sol" in text)]

mars_weather=long_texts[0]
print(mars_weather)

InSight sol 504 (2020-04-27) low -93.6ºC (-136.5ºF) high -6.7ºC (20.0ºF)
winds from the SW at 4.6 m/s (10.3 mph) gusting to 15.0 m/s (33.6 mph)
pressure at 6.80 hPa


In [19]:
#scrape Mars Facts table

url_mars_facts = "https://space-facts.com/mars/"

tables = pd.read_html(url_mars_facts)
df = tables[0]
df.columns = ["Mars Facts","Data"]
mars_fact_html_table = df.to_html()
#print(mars_fact_html_table)



In [20]:
#scrape Mars Hemispheres for pictures 
cerberus_url = "https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced"
schiaparelli_url = "https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced"
syrtis_url = "https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced"
valles_url = "https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced"

hemisphere_image_urls = []

def mars_hemispheres(hemi_url):
    browser = init_browser()
    
    browser.visit(hemi_url)

    time.sleep(5)

    hemi_html = browser.html
    hemi_soup = bs(hemi_html,"html.parser")

    

    image_results = hemi_soup.find("div",class_="downloads")
    image_url = image_results.find_all("li")[0].a.get("href")
    return image_url
    
hemisphere_image_urls.append({'title':'Valles Marineris Hemisphere', 'img_url': mars_hemispheres(valles_url)})   
hemisphere_image_urls.append({'title':'Cerberus Hemisphere', 'img_url': mars_hemispheres(cerberus_url)})
hemisphere_image_urls.append({'title':'Schiaparelli Hemisphere', 'img_url': mars_hemispheres(schiaparelli_url)})
hemisphere_image_urls.append({'title':'Syrtis Major Hemisphere', 'img_url': mars_hemispheres(syrtis_url)})
print(hemisphere_image_urls)

[{'title': 'Valles Marineris Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}, {'title': 'Cerberus Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}]
